<a href="https://colab.research.google.com/github/oscar-byte15/Adopt-Me-Ya-Front-End/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Diccionario de calles**
Crea un diccionario donde cada nombre de calle ***(key)*** tiene asignado un `id` ***(value)***

In [ ]:
import csv
import json

In [ ]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [ ]:
def printDictionary(dictionary):
    streets_pretty = json.dumps(dictionary, indent=2)
    print(streets_pretty)

In [ ]:
def cargarDatos(dictionary):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        
        for id, row in enumerate(reader):
            addKey_Values(dictionary, row[0].lower(), id)

In [ ]:
def createStreetsDictionary():   
    streets = {}
    cargarDatos(streets)

    return streets

In [ ]:
streetsDictionary = createStreetsDictionary()

##**Lista de Adyacencia de calles y sus intersecciones**




In [ ]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        id = streetsDictionary[streetName]
    else:
        id = -1 # la calle no está mapeada
    return id

In [ ]:
def printAdList(graph): 
    for i, intersection in enumerate(graph):
        print(i, intersection)
    
    print(f"Cantidad de vértices = {len(graph)}")

In [ ]:
# adjacencyList: arreglo de arreglos, donde la pos del primer elemento (1er arreglo)
# es la calle y el arreglo son todas las calles con las que se intersecta
def createAdjacencyList(adjacencyList):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        for row in reader:  
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))
            adjacencyList.append(intersections)

    return adjacencyList

In [ ]:
# creal la lista de adyacencia
intersectionsAdList = createAdjacencyList([])
printAdList(intersectionsAdList)

##**Lista de Adyacencia de intersecciones**

In [ ]:
def createKeys(graph):
    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            # si las keys aún no han sido creadas, las agrega al diccionario
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [ ]:
def addIntersections(graph, i, j):  
    street = graph[i][j]

    if street != -1:
        # agrego la intersección már próxima hacia la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego la intersección már próxima hacia la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [ ]:
def addIntersectionsReversed(graph, i, j):  
    street = graph[i][j]

    # ahora voy al reverso de la lista
    if i != -1 and street != -1:
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)
            
            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)
        
            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [ ]:
def joinIntersections(graph):
    n = len(graph)
    for i in range(n):        
        for j in range(len(graph[i])):
            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [ ]:
# calcula el total de nodos solo par el grafo "map"
def totalNodes(graph):
    total = 0 
    for key, value in graph.items():
        total += len(value)
        #print(key, ":", value)
    return total

In [ ]:
# grafo principal
map = createKeys(intersectionsAdList)
joinIntersections(intersectionsAdList)
print(f"Cantidad de nodos: {totalNodes(map)}")

##**Diccionario de Latitudes y Longitudes para cada intersección**

In [ ]:
def createLatLongDictionary(latlong):
    with open("Latitud y Longitud de calles.csv") as file: 
        reader = csv.reader(file)      
        # ignora el header (la 1era fila del csv)
        next(reader)
        for row in reader:
            # busco el id de la calle en mi diccionario de calles 
            intersection = streetExists(row[0]), streetExists(row[1])
            if intersection[1] != -1:
                latlong[intersection] = row[2], row[3]
    
    print(latlong)
    return latlong
      
latlong = createLatLongDictionary({})

###**Escribir archivos**

In [ ]:
def writeNodes():
    with open("nodes.txt", "w") as f:
        with open('Calles de San Martín de Porres.csv') as file: 
            reader = csv.reader(file)

            for row in reader:
                for i in range(1, len(row)):                    
                    f.write(f"{row[0]}, {row[i]}\n")
writeNodes()

In [ ]:
# Streets Dictionary
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")
writeStreetsDictionary()

In [ ]:
# Intersections Adjacency List
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])    
            f.write(', '.join(streets) + "\n")        
writeStreetsAdList()

In [ ]:
# Write street map
def writeStreetMap():
    with open("map.txt", "w") as f:  
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(str(key) + " " + listWOCommas + '\n')
writeStreetMap()

##**Aristas**

In [ ]:
from datetime import datetime
from math import radians, cos, sin, asin, sqrt

In [ ]:
# devuelve el tiempo actual automáticamente
def getCurrentTime():

    now = datetime.now()
    time = now.strftime("%H %p")
    time = time.split(" ")

    # convierto la hora a la timezone de Perú
    if time[0] >= "18":
        time[0] = int(time[0]) - 5
    
    return time
getCurrentTime()

[16, 'PM']

In [ ]:
def getlongitude(n1, n2):
     x1,y1=n1
     x2,y2=n2
     longitude =  math.sqrt((x2-x1)**2+(y2-y1)**2)
     return longitude

In [ ]:
def getFactor(time = '6'):
    factor = 0
    traffic_diccionary={
        "poco_trafico": 2,
        "trafico_promedio":4,
        "mucho_trafico":8
    }
    traffic_time = int(time)
    if(traffic_time in range (0,6) or traffic_time in range (19,25) ):
       factor = traffic_diccionary["poco_trafico"];

    elif(traffic_time in range (12,19)):
        factor = traffic_diccionary["trafico_promedio"];

    elif(traffic_time in range (6,10) or traffic_time in range (9,12)):
          factor = traffic_diccionary["mucho_trafico"];

    return factor;
  
getFactor()

8

In [ ]:
def formatearLatitudeorLongitude(input="-77.114.573"):
    separar = input
    separar = separar.split(".");

    union= separar[0]+"."+separar[1]+separar[2];
    variable_float = float(union)


    return variable_float

print(formatearLatitudeorLongitude())

-77.114573


In [ ]:
def calculateWeight(n1, n2):

    time = getCurrentTime()
    factor = getFactor(time)
    longitude = getlongitude(n1, n2)
    
    weight = factor * longitude

    return weight

In [ ]:
# el nuevo grafo será un diccionario, donde cada "key" contiene 1 arreglo de arreglos
# cada arreglo dentro del principal, tiene solo 2 posiciones
# 1era posición del arreglo: 1 tuple de intersecciones
# 2da posición del arreglo: peso de la arista con respecto al valor de la "key"

def updateGraph():
    # map con pesos
    mapW = {}
    for key in map:
        mapW[key] = []
        for arista in map[key]:
            weight = calculateWeight(key, arista)
            mapW[key].append([arista, weight])